In [1]:
# Importa los modulos necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

sns.set()

Objetivo: El dataset de incumplimiento de clientes para una compañía de tarjetas de crédito. De cada cliente se tiene el balance que le llegó de la tarjeta, si es un estudiante universitario o no y el sueldo anual. Además, se cuenta si el cliente incumplió el pago o no.

In [2]:
# Carga el archivo con los datos procesados
credit = pd.read_csv("./datasets/UCI_Credit_Card_procesados.csv")

In [4]:
credit.head()

,ID,PAY_0,default.payment.next.month
0,-1.733014,1.914456,1.886954
1,-1.732899,-0.899921,1.886954
2,-1.732783,0.038205,-0.529937
3,-1.732668,0.038205,-0.529937
4,-1.732552,-0.899921,-0.529937


In [8]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29859 entries, 0 to 29858
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          29859 non-null  float64
 1   PAY_0                       29859 non-null  float64
 2   default.payment.next.month  29859 non-null  float64
dtypes: float64(3)
memory usage: 699.9 KB


In [9]:
# Convertir la columna de float64 a int64
credit['default.payment.next.month'] = credit['default.payment.next.month'].astype('int64')

In [10]:
# Separa variables predictoras de la variable a predecir
X = credit.loc[:, ['PAY_0']].values
y = credit.loc[:, "default.payment.next.month"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify=y)

In [11]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train) 
X_test = sc_X.transform(X_test)

Modelo árbol de clasificación

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Crear el modelo de árbol de decisión
tree_regressor = DecisionTreeClassifier()

# Definir la grilla de hiperparámetros
param_grid = [
    {"criterion": ["entropy"],
     "splitter": ["best"],
     "max_depth": [x for x in range(10)],
     "min_samples_split": [x for x in range(20)],
     "min_samples_leaf": [x for x in range(10)],
     "max_features": [None],
     "ccp_alpha": [x * 0.01 for x in range(0, 50)]}
]

# Configurar GridSearchCV
grid_search = GridSearchCV(tree_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', refit=True)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_


c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
135500 fits failed out of a total of 500000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50000 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\skle

Mejores parámetros encontrados:
{'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': 1, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan ... -0.21925516 -0.21925516
 -0.21925516]
  warnings.warn(


In [13]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

y_pred_test = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

print(f"La exactitud de clasificación es: {accuracy}")
print(f"F1-score de testeo fue: {f1}")
print(f"Precisión de testeo fue: {precision}")
print(f"Recall de testeo fue: {recall}")

La exactitud de clasificación es: 0.8187541862022772
F1-score de testeo fue: 0.436484798000833
Precisión de testeo fue: 0.6858638743455497
Recall de testeo fue: 0.32009773976786804
